In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import SVC
from joblib import dump,load
import os
from sklearn.metrics import accuracy_score, classification_report
def fetch_data(ticker):
    stock = yf.Ticker(ticker)
    data = stock.history(period="max")  # Fetch 5 years of data
    data.reset_index(inplace=True)
    return data


In [2]:
stock_list=pd.read_csv('nasdaq-listed.csv')

In [3]:

def preprocess_data(data):
    # Feature engineering: Create relevant columns
    data['open-close'] = data['Open'] - data['Close']
    data['high-low'] = data['High'] - data['Low']
    data['price-change'] = data['Close'].pct_change()
    data['is_quarter_end'] = np.where((data['Date'].dt.month % 3 == 0) & (data['Date'].dt.day > 23), 1, 0)
    data['SMA_10'] = data['Close'].rolling(window=10).mean()
    data['SMA_50'] = data['Close'].rolling(window=50).mean()
    data['SMA_200'] = data['Close'].rolling(window=200).mean()
    data['EMA_10'] = data['Close'].ewm(span=10, adjust=False).mean()
    data['EMA_50'] = data['Close'].ewm(span=50, adjust=False).mean()
    data['EMA_200'] = data['Close'].ewm(span=200, adjust=False).mean()
    # Drop unnecessary columns
    data = data.drop(['Dividends', 'Stock Splits'], axis=1, errors='ignore')
    data.dropna(inplace=True)  # Handle missing values
    
    return data




In [4]:
accuracy_list=pd.read_csv("Accuracy_Data_NASDAQ.csv")

print(accuracy_list)

      STOCK   Accuracy Symbol
0       NaN  55.911330   AACG
1       NaN  54.248366   AADI
2       NaN  53.197674   AADR
3       NaN  49.354839    AAL
4       NaN  67.598017   AAME
...     ...        ...    ...
3759    NaN  62.280306   USEG
3760    NaN  46.875000   USFI
3761    NaN  58.695652   USGO
3762    NaN  53.810624   USIG
3763    NaN  70.919176   USIO

[3764 rows x 3 columns]


In [7]:
for index, row in stock_list.iloc[4428:].iterrows():
    stock=row['Stocks']
    data=fetch_data(stock)
    if (not data.empty) & (len(data)>11):
        data=preprocess_data(data)
        model_dir=f"./models/{stock}/"
        if not os.path.exists(model_dir):
                    os.mkdir(model_dir)
        accuracy_list.to_csv("Accuracy_Data_NASDAQ.csv",index=False)
        # Define features (X) and target (y)
        features = data[['open-close', 'high-low', 'Volume', 'is_quarter_end','SMA_10','SMA_50',"SMA_200",'EMA_10','EMA_50',"EMA_200"]]
        target = np.where(data['Close'].shift(-1) > data['Close'], 1, 0)  # 1 foSr price increase, 0 otherwise
        if features.shape[0] > 0 and target.shape[0] > 0:
   
            X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
            
            # Standardize features
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            # Initialize and train the model
            model = SVC(kernel='poly', probability=True, random_state=42)  
            model.fit(X_train, y_train)
            
            # Save the scaler and model for reuse
            
            dump(scaler, f"{model_dir}{stock}_scaler.joblib")
            dump(model, f"{model_dir}{stock}_predictor.joblib")
            y_pred = model.predict(X_test)
            
            # Evaluate performance
            print(stock," ",index)
            print("Accuracy:", accuracy_score(y_test, y_pred))
            print("Classification Report:\n", classification_report(y_test, y_pred))
            stock_accuracy=pd.DataFrame({"Symbol":[row['Stocks']],"Accuracy":[accuracy_score(y_test,y_pred)*100]})
            accuracy_list=pd.concat([accuracy_list,stock_accuracy])
            print("Accuracy List Length :",len(accuracy_list))
        

USTB   4429
Accuracy: 0.5527950310559007
Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.02      0.04       144
           1       0.55      0.98      0.71       178

    accuracy                           0.55       322
   macro avg       0.53      0.50      0.37       322
weighted avg       0.53      0.55      0.41       322

Accuracy List Length : 3768
USVM   4430
Accuracy: 0.48757763975155277
Classification Report:
               precision    recall  f1-score   support

           0       0.18      0.01      0.02       158
           1       0.50      0.95      0.65       164

    accuracy                           0.49       322
   macro avg       0.34      0.48      0.34       322
weighted avg       0.34      0.49      0.34       322

Accuracy List Length : 3769
USVN   4431
Accuracy: 0.4
Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.03      0.06        30


VACHW: Period 'max' is invalid, must be one of ['1d', '5d']


VALN   4447
Accuracy: 0.5655172413793104
Classification Report:
               precision    recall  f1-score   support

           0       0.57      0.93      0.70        81
           1       0.54      0.11      0.18        64

    accuracy                           0.57       145
   macro avg       0.55      0.52      0.44       145
weighted avg       0.56      0.57      0.47       145

Accuracy List Length : 3783
VALU   4448
Accuracy: 0.5975728155339806
Classification Report:
               precision    recall  f1-score   support

           0       0.60      0.98      0.75      1232
           1       0.49      0.02      0.04       828

    accuracy                           0.60      2060
   macro avg       0.54      0.50      0.39      2060
weighted avg       0.55      0.60      0.46      2060

Accuracy List Length : 3784
VANI   4449
Accuracy: 0.5808510638297872
Classification Report:
               precision    recall  f1-score   support

           0       0.58      1.00      0

VCICW: Period 'max' is invalid, must be one of ['1d', '5d']


VCIG   4458
Accuracy: 0.574468085106383
Classification Report:
               precision    recall  f1-score   support

           0       0.57      1.00      0.73        27
           1       0.00      0.00      0.00        20

    accuracy                           0.57        47
   macro avg       0.29      0.50      0.36        47
weighted avg       0.33      0.57      0.42        47

Accuracy List Length : 3791


...

ZS   4785
Accuracy: 0.5742574257425742
Classification Report:
               precision    recall  f1-score   support

           0       0.48      0.09      0.15       128
           1       0.58      0.93      0.72       175

    accuracy                           0.57       303
   macro avg       0.53      0.51      0.43       303
weighted avg       0.54      0.57      0.48       303

Accuracy List Length : 4068
ZTEK   4786
Accuracy: 0.627906976744186
Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.97      0.77       385
           1       0.23      0.01      0.03       217

    accuracy                           0.63       602
   macro avg       0.43      0.49      0.40       602
weighted avg       0.49      0.63      0.50       602

Accuracy List Length : 4069
ZUMZ   4787
Accuracy: 0.5221052631578947
Classification Report:
               precision    recall  f1-score   support

           0       0.52      0.79      0.63

ZVZZT: Period 'max' is invalid, must be one of ['1d', '5d']
ZWZZT: Period 'max' is invalid, must be one of ['1d', '5d']
$ZXYZ.A: possibly delisted; no timezone found
ZXZZT: Period 'max' is invalid, must be one of ['1d', '5d']


ZYME   4795
Accuracy: 0.4783861671469741
Classification Report:
               precision    recall  f1-score   support

           0       0.48      0.89      0.62       166
           1       0.50      0.10      0.17       181

    accuracy                           0.48       347
   macro avg       0.49      0.50      0.40       347
weighted avg       0.49      0.48      0.39       347

Accuracy List Length : 4074
ZYXI   4796
Accuracy: 0.6449060336300693
Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.99      0.78       657
           1       0.31      0.01      0.02       354

    accuracy                           0.64      1011
   macro avg       0.48      0.50      0.40      1011
weighted avg       0.53      0.64      0.52      1011

Accuracy List Length : 4075


$FILE CREATION TIME: 1129202421:31: possibly delisted; no timezone found


AttributeError: 'float' object has no attribute 'upper'

In [ ]:
accuracy_list.to_csv("Accuracy_Data_NASDAQ.csv",index=False)